# Quick Start: Belavkin Optimizer

This notebook demonstrates the basic usage of the Belavkin optimizer on a simple modular arithmetic task.

## Overview

The Belavkin optimizer implements a novel update rule inspired by quantum filtering:

$$\theta_{t+1} = \theta_t - [\gamma(\nabla L(\theta))^2 + \eta\nabla L(\theta)]\Delta t + \beta\nabla L(\theta)\sqrt{\Delta t}\epsilon$$

where:
- $\gamma$: Damping factor (measurement backaction)
- $\eta$: Learning rate
- $\beta$: Exploration factor
- $\epsilon \sim N(0,1)$: Gaussian noise

In [ ]:
# Imports
import sys
sys.path.insert(0, '..')

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from belavkin_ml.optimizer import BelavkinOptimizer, AdaptiveBelavkinOptimizer
from belavkin_ml.datasets.synthetic import ModularArithmeticDataset, create_dataloaders

sns.set_style('whitegrid')
%matplotlib inline

## 1. Create a Simple Dataset

We'll use modular addition: $f(x, y) = (x + y) \mod p$

In [ ]:
# Create modular arithmetic dataset
p = 97  # Prime modulus
dataset = ModularArithmeticDataset(
    p=p,
    operation='addition',
    train_fraction=0.5,
    seed=42
)

# Get dataset info
info = dataset.get_info()
print(f"Dataset Information:")
print(f"  Modulus: {info['p']}")
print(f"  Operation: {info['operation']}")
print(f"  Training examples: {info['train_examples']}")
print(f"  Test examples: {info['test_examples']}")
print(f"  Input dimension: {info['input_dim']}")
print(f"  Output dimension: {info['output_dim']}")

In [ ]:
# Create data loaders
train_loader, test_loader = create_dataloaders(
    dataset,
    batch_size=512,
    num_workers=0
)

print(f"Number of training batches: {len(train_loader)}")
print(f"Number of test batches: {len(test_loader)}")

## 2. Define a Simple MLP Model

In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
    
    def forward(self, x):
        return self.net(x)

# Create model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SimpleMLP(
    input_dim=info['input_dim'],
    output_dim=info['output_dim'],
    hidden_dim=128
).to(device)

print(f"Model: {sum(p.numel() for p in model.parameters())} parameters")
print(f"Device: {device}")

## 3. Create Belavkin Optimizer

Let's compare the Belavkin optimizer with Adam.

In [ ]:
# Training function
def train_epoch(model, optimizer, criterion, train_loader, device):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    
    return total_loss / total, correct / total

def evaluate(model, criterion, test_loader, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            total_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    return total_loss / total, correct / total

In [ ]:
# Training loop
def train_model(model, optimizer, criterion, train_loader, test_loader, n_epochs, device):
    train_losses = []
    train_accs = []
    test_losses = []
    test_accs = []
    
    for epoch in range(n_epochs):
        train_loss, train_acc = train_epoch(model, optimizer, criterion, train_loader, device)
        test_loss, test_acc = evaluate(model, criterion, test_loader, device)
        
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        test_losses.append(test_loss)
        test_accs.append(test_acc)
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{n_epochs} - "
                  f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
                  f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
    
    return {
        'train_losses': train_losses,
        'train_accs': train_accs,
        'test_losses': test_losses,
        'test_accs': test_accs
    }

## 4. Train with Belavkin Optimizer

In [ ]:
# Reset model
model_belavkin = SimpleMLP(
    input_dim=info['input_dim'],
    output_dim=info['output_dim'],
    hidden_dim=128
).to(device)

# Create Belavkin optimizer
optimizer_belavkin = BelavkinOptimizer(
    model_belavkin.parameters(),
    lr=1e-3,
    gamma=1e-4,
    beta=1e-2,
)

criterion = nn.CrossEntropyLoss()

print("Training with Belavkin Optimizer...")
results_belavkin = train_model(
    model_belavkin,
    optimizer_belavkin,
    criterion,
    train_loader,
    test_loader,
    n_epochs=50,
    device=device
)

## 5. Train with Adam (Baseline)

In [ ]:
# Reset model
model_adam = SimpleMLP(
    input_dim=info['input_dim'],
    output_dim=info['output_dim'],
    hidden_dim=128
).to(device)

# Create Adam optimizer
optimizer_adam = torch.optim.Adam(
    model_adam.parameters(),
    lr=1e-3,
)

print("Training with Adam Optimizer...")
results_adam = train_model(
    model_adam,
    optimizer_adam,
    criterion,
    train_loader,
    test_loader,
    n_epochs=50,
    device=device
)

## 6. Compare Results

In [ ]:
# Plot learning curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Test accuracy
axes[0].plot(results_belavkin['test_accs'], label='Belavkin', linewidth=2)
axes[0].plot(results_adam['test_accs'], label='Adam', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Test Accuracy')
axes[0].set_title('Test Accuracy Comparison')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Test loss
axes[1].plot(results_belavkin['test_losses'], label='Belavkin', linewidth=2)
axes[1].plot(results_adam['test_losses'], label='Adam', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Test Loss')
axes[1].set_title('Test Loss Comparison')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal Results:")
print(f"Belavkin - Best Test Acc: {max(results_belavkin['test_accs']):.4f}")
print(f"Adam     - Best Test Acc: {max(results_adam['test_accs']):.4f}")

## 7. Try Adaptive Belavkin Optimizer

The adaptive variant automatically tunes hyperparameters.

In [ ]:
# Reset model
model_adaptive = SimpleMLP(
    input_dim=info['input_dim'],
    output_dim=info['output_dim'],
    hidden_dim=128
).to(device)

# Create Adaptive Belavkin optimizer
optimizer_adaptive = AdaptiveBelavkinOptimizer(
    model_adaptive.parameters(),
    lr=1e-3,
    gamma=1e-4,
    beta=1e-2,
    adapt_lr=True,
    adapt_gamma=True,
    adapt_beta=True,
)

print("Training with Adaptive Belavkin Optimizer...")
results_adaptive = train_model(
    model_adaptive,
    optimizer_adaptive,
    criterion,
    train_loader,
    test_loader,
    n_epochs=50,
    device=device
)

In [ ]:
# Plot all three
plt.figure(figsize=(10, 6))
plt.plot(results_belavkin['test_accs'], label='Belavkin', linewidth=2)
plt.plot(results_adam['test_accs'], label='Adam', linewidth=2)
plt.plot(results_adaptive['test_accs'], label='Adaptive Belavkin', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Test Accuracy')
plt.title('Optimizer Comparison on Modular Addition')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nFinal Comparison:")
print(f"Belavkin         - Best: {max(results_belavkin['test_accs']):.4f}")
print(f"Adam             - Best: {max(results_adam['test_accs']):.4f}")
print(f"Adaptive Belavkin - Best: {max(results_adaptive['test_accs']):.4f}")

## 8. Inspect Adaptive Parameters

The adaptive optimizer tracks how hyperparameters change during training.

In [ ]:
# Get adaptation statistics
stats = optimizer_adaptive.get_adaptation_stats()

print("Adaptation Statistics:")
print(f"\nCurrent Learning Rate: {stats['current_params']['group_0']['lr']:.6f}")
print(f"Current Gamma: {stats['current_params']['group_0']['gamma']:.6f}")
print(f"Current Beta: {stats['current_params']['group_0']['beta']:.6f}")

# Plot gradient norm history
if stats['grad_norm_history']:
    plt.figure(figsize=(10, 4))
    plt.plot(stats['grad_norm_history'])
    plt.xlabel('Step')
    plt.ylabel('Gradient Norm')
    plt.title('Gradient Norm During Training')
    plt.grid(True, alpha=0.3)
    plt.show()

## Next Steps

1. **Run Full Benchmarks**: Use `experiments/track1_optimizer/run_modular_arithmetic.py`
2. **Try Different Tasks**: Sparse parity, modular composition
3. **Hyperparameter Tuning**: Search over gamma, beta ranges
4. **Analyze Results**: Use visualization tools in `belavkin_ml.utils`
5. **Scale Up**: Try on MNIST, CIFAR-10

See the main README for more details!